# Netmiko & Diffing

In [1]:
HOST = '192.75.232.222'
PORT_NC = 830
PORT_SSH = 22
USER = 'cisco'
PASS = 'cisco'
PLATFORM = 'cisco_xr'

## Connect both netmiko and ncclient

In [2]:
from netmiko import ConnectHandler
from ncclient import manager
from lxml import etree


def pretty_print(retval):
    print(etree.tostring(retval.data, pretty_print=True))

def my_unknown_host_cb(host, fingerprint):
    return True

def get_reply(chan, eom):
    bytes = u''
    while bytes.find(eom)==-1:
        bytes += chan.recv(65535).decode('utf-8')
    return bytes

m = manager.connect(host=HOST, port=PORT_NC, username=USER, password=PASS,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)
d = ConnectHandler(device_type=PLATFORM, ip=HOST, port=PORT_SSH, username=USER, password=PASS)

prompt = d.find_prompt()

## Sample Config To Apply

In [3]:
config = '''interface Loopback666
description Hi
commit
end
'''

## Capture Starting Config

In [4]:
running_config_before_text = d.send_command('show running-config')
running_config_before_xml = etree.tostring(m.get_config(source='running').data)

## Apply Config

In [5]:
output = d.send_config_set(config.splitlines())

## Capture Post-Configlet Running Config

In [6]:
running_config_after_text = d.send_command('show running-config')
running_config_after_xml = etree.tostring(m.get_config(source='running').data)

## XML Diff In Text Form

In [7]:
from difflib import context_diff

#
# remember to skip the first few lines that have timestamps & stuff that may differ
#
print('\n'.join(context_diff(running_config_before_xml.splitlines(),
                             running_config_after_xml.splitlines())))

## Rollback Using CLI

In [8]:
rollback_output = d.send_command('rollback configuration last 1')

In [9]:
rollback_output

u'\nWed Mar 29 12:23:43.691 UTC\n\nLoading Rollback Changes.\nLoaded Rollback Changes in 1 sec \nCommitting.\n3 items committed in 1 sec (2)items/sec\nUpdating.\nUpdated Commit database in 1 sec \nConfiguration successfully rolled back 1 commits.'

## Calculate Reapply XML Document

We also exclude certain data (e.g. that from Calvados).

In [10]:
EXCL_TAGS = [
    '{http://cisco.com/ns/yang/Cisco-IOS-XR-man-netconf-cfg}netconf-yang',
    '{http://cisco.com/ns/yang/Cisco-IOS-XR-crypto-sam-cfg}crypto',
    '{http://cisco.com/calvados/fpd}fpd',
    '{http://tail-f.com/ns/aaa/1.1}aaa',
    '{http://cisco.com/ns/yang/Cisco-IOS-XR-aaa-lib-cfg}aaa',
    '{http://tail-f.com/ns/confd_dyncfg/1.0}confdConfig',
    '{http://tail-f.com/ns/mibs/SNMP-COMMUNITY-MIB/200308060000Z}SNMP-COMMUNITY-MIB',
    '{http://tail-f.com/ns/mibs/SNMP-NOTIFICATION-MIB/200210140000Z}SNMP-NOTIFICATION-MIB',
    '{http://tail-f.com/ns/mibs/SNMP-TARGET-MIB/200210140000Z}SNMP-TARGET-MIB',
    '{http://tail-f.com/ns/mibs/SNMP-USER-BASED-SM-MIB/200210160000Z}SNMP-USER-BASED-SM-MIB',
    '{http://tail-f.com/ns/mibs/SNMP-VIEW-BASED-ACM-MIB/200210160000Z}SNMP-VIEW-BASED-ACM-MIB',
    '{http://tail-f.com/ns/mibs/SNMPv2-MIB/200210160000Z}SNMPv2-MIB',
    '{http://www.cisco.com/ns/yang/Cisco-IOS-XR-sysadmin-services}service',
    '{http://www.cisco.com/panini/calvados/sdr_mgr/sdr_sm}sdr-config',
    '{http://www.cisco.com/panini/calvados/sdr_mgr/sdr_sm}private-sdr',
    '{http://openconfig.net/yang/local-routing}local-routes',
    '{http://openconfig.net/yang/interfaces}interfaces'
]
reapply = '<config>\n'
for item in etree.fromstring(running_config_after_xml).getchildren():
    if item.tag not in EXCL_TAGS:
        reapply = reapply + etree.tostring(item)
reapply = reapply + '</config>'
print(reapply)

<config>
<restconf xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-man-restconf-cfg" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
   <agent>
    <enable/>
   </agent>
  </restconf>
  <ip xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ip-tcp-cfg" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
   <cinetd>
    <services>
     <vrfs>
      <vrf>
       <vrf-name>default</vrf-name>
       <ipv4>
        <telnet>
         <tcp>
          <maximum-server>35</maximum-server>
         </tcp>
        </telnet>
       </ipv4>
      </vrf>
     </vrfs>
    </services>
   </cinetd>
  </ip>
  <web xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-man-http-lighttpd-yang-cfg" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
   <server>
    <service>
     <restconf>
      <enable/>
      <http-enable/>
     </restconf>
    </service>
   </server>
  </web>
  <router-static xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ip-static-cfg" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
   <default-vrf

## Reapply Netconf Config

In [11]:
m.edit_config(reapply, format='xml', target='candidate', default_operation='merge')
m.commit()

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:03e4213d-3c12-4140-b65f-ca1870bba87b" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <ok/>
</rpc-reply>

## Get Running Config Again

In [12]:
running_config_after_xml_apply_text = d.send_command('show running-config')
rollback_output = d.send_command('rollback configuration last 1')
print(rollback_output)


Wed Mar 29 12:24:12.502 UTC

Loading Rollback Changes.
Loaded Rollback Changes in 1 sec 
Committing.
19 items committed in 1 sec 
Updating.
Updated Commit database in 1 sec 
Configuration successfully rolled back 1 commits.


## Diff After Original Apply & After XML Apply (Text Only)

In [13]:
from difflib import context_diff

#
# remember to skip the first few lines that have timestamps & stuff that may differ
#
print('\n'.join(context_diff(running_config_after_text.splitlines()[5:],
                             running_config_after_xml_apply_text.splitlines()[5:])))

## Tidyup Sessions

In [14]:
d.disconnect()

In [15]:
m.close_session()

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:f33ff400-5321-44bd-87d5-e202e76b4403" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <ok/>
</rpc-reply>